In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb # <-- Importamos el nuevo algoritmo
import os
import sys

# Añadimos la carpeta principal al path para encontrar nuestros módulos
sys.path.insert(0, os.path.abspath('..'))

# Importamos TODAS nuestras funciones de ingeniería de características
from src.feature_engineering import add_pythagorean_expectation, EloRatingSystem, add_elo_ratings, add_rolling_win_percentage

print("--- Fase 3: Entrenamiento con Algoritmo Profesional (XGBoost) ---")

# 1. Cargar el NUEVO dataset enriquecido
csv_path = os.path.join('..', 'data', 'historical_games_rich.csv')
df = pd.read_csv(csv_path, parse_dates=['game_date']) # parse_dates convierte la columna de fecha
print(f"Dataset enriquecido cargado con {df.shape} filas.")

# 2. Aplicar TODA la ingeniería de características (ahora con datos reales)
df_v4 = add_pythagorean_expectation(df)
df_v4 = add_rolling_win_percentage(df_v4)

# 3. Entrenar el sistema Elo con datos reales
elo_sys = EloRatingSystem()
df_v4['winner'] = df_v4.apply(lambda row: row['h_team_name'] if row['target'] == 1 else row['v_team_name'], axis=1)
df_v4['loser'] = df_v4.apply(lambda row: row['v_team_name'] if row['target'] == 1 else row['h_team_name'], axis=1)

for index, row in df_v4.sort_values('game_date').iterrows():
    elo_sys.update_ratings(row['winner'], row['loser'])

df_v4 = add_elo_ratings(df_v4, elo_sys)

# 4. Preparar los datos para el modelo v4
y = df_v4['target']
features_v4 = ['h_team_wins_season', 'h_team_losses_season', 'v_team_wins_season', 'v_team_losses_season', 
               'pythag_diff', 'elo_diff', 'win_pct_roll_diff']
X = df_v4[features_v4]

print("\nCaracterísticas finales para el modelo XGBoost:")
print(X.columns.tolist())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. Entrenar y Evaluar el modelo XGBoost
print("\nEntrenando modelo XGBoost...")
# Usamos parámetros iniciales comunes para XGBoost
model_v4 = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    use_label_encoder=False,
    eval_metric='logloss'
)
model_v4.fit(X_train, y_train)
print("¡Modelo XGBoost (v4) entrenado!")

predictions_v4 = model_v4.predict(X_test)
accuracy_v4 = accuracy_score(y_test, predictions_v4)

print("\n" + "="*60)
print(f"  PRECISIÓN RandomForest (v3): 0.6190")
print(f"  PRECISIÓN XGBoost (v4): {accuracy_v4:.4f}")
print("="*60)

improvement_v4 = accuracy_v4 - 0.6190
print(f"\n¡Mejora por cambiar de algoritmo: {improvement_v4:+.2%}!")
total_improvement = accuracy_v4 - 0.5585
print(f"¡Mejora total sobre la línea base: {total_improvement:+.2%}!")

# 6. Guardar el modelo XGBoost (este será nuestro nuevo campeón)
# assets_dir = os.path.join('..', 'src', 'assets')
# model_path_v4 = os.path.join(assets_dir, 'mlb_model_xgb.joblib')
# joblib.dump(model_v4, model_path_v4)
# print(f"Modelo XGBoost guardado en '{model_path_v4}'")

--- Fase 3: Entrenamiento con Algoritmo Profesional (XGBoost) ---
Dataset enriquecido cargado con (2476, 8) filas.
--- Aplicando Ingeniería de Características: Expectativa Pitagórica ---
¡Característica 'pythag_diff' creada con éxito!
--- Aplicando Ingeniería de Características: Forma Reciente (Medias Móviles) ---
¡Característica 'win_pct_roll_diff' creada con éxito!
--- Aplicando Ingeniería de Características: Calificaciones Elo ---
¡Característica 'elo_diff' creada con éxito!

Características finales para el modelo XGBoost:
['h_team_wins_season', 'h_team_losses_season', 'v_team_wins_season', 'v_team_losses_season', 'pythag_diff', 'elo_diff', 'win_pct_roll_diff']

Entrenando modelo XGBoost...
¡Modelo XGBoost (v4) entrenado!

  PRECISIÓN RandomForest (v3): 0.6190
  PRECISIÓN XGBoost (v4): 0.6411

¡Mejora por cambiar de algoritmo: +2.21%!
¡Mejora total sobre la línea base: +8.26%!


C:\proyectos\predictor_mlb\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [09:37:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
